In [3]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
API_KEY = os.environ.get("API_KEY")

In [125]:
bibleVersionID = "de4e12af7f28f599-02"

books_res = requests.get(
    url=f"https://api.scripture.api.bible/v1/bibles/{bibleVersionID}/books",
    headers={
        "api-key": API_KEY,
        "Accept-Encoding": "application/json",
    },
)

In [128]:
books_json = books_res.json()
books = []
for book in books_json["data"]:
    books.append(book["id"])

In [155]:
chapters = []
for book in books:
    chapters_res = requests.get(
        url=f"https://api.scripture.api.bible/v1/bibles/{bibleVersionID}/books/{book}/chapters",
        headers={
            "api-key": API_KEY,
            "Accept-Encoding": "application/json",
        },
    )

    for chapter in chapters_res.json()["data"]:
        chapters.append(chapter["id"])

In [165]:
verse_ids = []
for chapter in chapters:
    verses_res = requests.get(
        url=f"https://api.scripture.api.bible/v1/bibles/{bibleVersionID}/chapters/{chapter}/verses",
        headers={
            "api-key": API_KEY,
            "Accept-Encoding": "application/json",
        },
    )

    for verse in verses_res.json()["data"]:
        verse_ids.append(verse["id"])

In [175]:
verse_ids_no_intro = filter(lambda x: "intro" not in x, verse_ids)
len(list(verse_ids_no_intro))

31102

In [280]:
chapters_no_intro = filter(lambda x: "intro" not in x, chapters)
chapters_no_intro = list(chapters_no_intro)

In [232]:
import pandas

In [255]:
for idx, chapter in enumerate(chapters_no_intro):
    chapter_verses_res = requests.get(
        url=f"https://api.scripture.api.bible/v1/bibles/{bibleVersionID}/chapters/{chapter}",
        headers={
            "api-key": API_KEY,
            "Accept-Encoding": "application/json",
        },
        params={
            "content-type": "text",
            "include-notes": "false",
            "include-titles": "false",
        },
    )

    df = pandas.DataFrame(
        {
            "index": idx,
            "chapter": chapter,
            "verses": chapter_verses_res.json()["data"]["content"],
        },
        index=[0],
    )
    df.to_csv("bible_verses.csv", mode="a", header=False, index=False)

In [200]:
import re

In [273]:
df = pandas.read_csv("bible_verses.csv", index_col=0)

In [298]:
chapter_verses = []
for idx, verse in enumerate(df["verses"]):
    verse_clean = re.split(r"\s*\[\d+\]\s*", verse.strip())
    verse_clean = [item for item in verse_clean if item]
    for idx_verse, verse in enumerate(verse_clean):
        chapter_verses.append(
            {"citation": f"{df['chapter'][idx]}.{idx_verse+1}", "verse": verse}
        )

In [293]:
df_verses = pandas.DataFrame(chapter_verses)
df_verses.to_csv("bible_verses_clean.csv", index=False)